IMPORTING LIBRARIES


In [13]:
import csv
import math


LOAD PRODUCTS FROM CSV

In [14]:
def load_products_from_csv(filename):
    products = []
    with open(filename, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            products.append([
                int(row['id']),
                row['name'],
                row['category'],
                float(row['price']),
                float(row['rating'])
            ])
    return products

NORMALIZE PRODUCTS

In [15]:
def normalize_products(products):
    prices = [p[3] for p in products]
    ratings = [p[4] for p in products]
    min_price, max_price = min(prices), max(prices)
    min_rating, max_rating = min(ratings), max(ratings)
    normalized = [
        [p[0], p[1], p[2],
         (p[3] - min_price) / (max_price - min_price),
         (p[4] - min_rating) / (max_rating - min_rating)]
        for p in products
    ]
    return normalized, (min_price, max_price, min_rating, max_rating)

def normalize_user_input(user_input, norm_params):
    min_price, max_price, min_rating, max_rating = norm_params
    return [
        (user_input[0] - min_price) / (max_price - min_price),
        (user_input[1] - min_rating) / (max_rating - min_rating)
    ]

EUCLIDEAN DISTANCE

In [4]:
def euclidean_distance(p1, p2):
    return math.sqrt(sum((a - b)**2 for a, b in zip(p1, p2)))

WEIGHTED KNN

In [5]:
def knn_weighted(products, user_input, k=3):
    norm_data, norm_params = normalize_products(products)
    norm_input = normalize_user_input(user_input, norm_params)
    distances = []
    for p in norm_data:
        d = euclidean_distance([p[3], p[4]], norm_input)
        distances.append((d, p[2]))
    distances.sort()
    votes = {}
    for i in range(min(k, len(distances))):
        dist, cat = distances[i]
        weight = 1 / (dist + 1e-5)
        votes[cat] = votes.get(cat, 0) + weight
    return max(votes, key=votes.get)


NAIVE BAYES

In [16]:
def gaussian_pdf(x, mean, std):
    if std == 0: return 0
    exponent = math.exp(-((x - mean) ** 2) / (2 * std**2))
    return (1 / (math.sqrt(2 * math.pi) * std)) * exponent

def naive_bayes(products, user_input):
    categories = {}
    for p in products:
        cat = p[2]
        if cat not in categories:
            categories[cat] = {'prices': [], 'ratings': []}
        categories[cat]['prices'].append(p[3])
        categories[cat]['ratings'].append(p[4])
    probs = {}
    for cat, vals in categories.items():
        mean_price = sum(vals['prices']) / len(vals['prices'])
        std_price = math.sqrt(sum((x - mean_price)**2 for x in vals['prices']) / len(vals['prices']))
        mean_rating = sum(vals['ratings']) / len(vals['ratings'])
        std_rating = math.sqrt(sum((x - mean_rating)**2 for x in vals['ratings']) / len(vals['ratings']))
        p_price = gaussian_pdf(user_input[0], mean_price, std_price)
        p_rating = gaussian_pdf(user_input[1], mean_rating, std_rating)
        probs[cat] = p_price * p_rating
    return max(probs, key=probs.get)

BFS

In [7]:
def bfs_sorted(products, target_category, user_budget, sort_by="price"):
    matched = [p for p in products if p[2] == target_category]
    if not matched:
        return None
    if sort_by == "price":
        matched.sort(key=lambda p: abs(p[3] - user_budget))
    elif sort_by == "rating":
        matched.sort(key=lambda p: -p[4])
    return matched

MAIN FUNCTION

In [18]:
def main():
    filename = "/content/product.csv"
    products = load_products_from_csv(filename)

    print("== AI Virtual Shopping Assistant ==")
    budget = float(input("Enter your budget: "))
    rating = float(input("Enter your preferred rating: "))
    user_input = [budget, rating]

    knn_cat = knn_weighted(products, user_input)
    nb_cat = naive_bayes(products, user_input)

    bfs_knn_products = bfs_sorted(products, knn_cat, budget)
    bfs_nb_products = bfs_sorted(products, nb_cat, budget)

    print("\n=== PREDICTIONS ===")
    print("\n1. [KNN Weighted] Category:", knn_cat)
    print("\n2. [Naive Bayes] Category:", nb_cat)


    if bfs_knn_products:
        print(f"\n3. [BFS Sorted using KNN] Top Products in '{knn_cat}':")
        for i, p in enumerate(bfs_knn_products[:3], 1):
            print(f"   {i}. {p[1]} - ₹{p[3]}, ⭐ {p[4]}")
    else:
        print("\n3. No matching products found in category.")

    if bfs_nb_products:
        print(f"\n4. [BFS Sorted using NB] Top Products in '{nb_cat}':")
        for i, p in enumerate(bfs_nb_products[:3], 1):
            print(f"   {i}. {p[1]} - ₹{p[3]}, ⭐ {p[4]}")
    else:
        print("\n4. No matching products found in category.")


# === Run ===
if __name__ == "__main__":
    main()


== AI Virtual Shopping Assistant ==
Enter your budget: 5000
Enter your preferred rating: 4

=== PREDICTIONS ===

1. [KNN Weighted] Category: Headphones

2. [Naive Bayes] Category: Smartphone

3. [BFS Sorted using KNN] Top Products in 'Headphones':
   1. Sennheiser HD 450BT - ₹5672.0, ⭐ 4.2
   2. Boat Rockerz 550 - ₹8410.0, ⭐ 3.8
   3. JBL Quantum 800 - ₹12496.0, ⭐ 4.5

4. [BFS Sorted using NB] Top Products in 'Smartphone':
   1. OnePlus Nord 2T - ₹4467.0, ⭐ 4.7
   2. Redmi Note 11 - ₹5639.0, ⭐ 4.6
   3. Realme GT - ₹7315.0, ⭐ 4.6
